#### Importing required libraries

In [198]:
import pandas as pd
import numpy as np
from functools import reduce
from fbprophet import Prophet
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [199]:
pwd

'C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries'

#### Reading input datasets

In [218]:
cust_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_customers_dataset.csv')
geo_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_geolocation_dataset.csv')
orders_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_orders_dataset.csv')
order_item_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_order_items_dataset.csv')
order_payments_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_order_payments_dataset.csv')
order_reviews_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_order_reviews_dataset.csv')
product_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_products_dataset.csv')
sellers_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\olist_sellers_dataset.csv')
category_data = pd.read_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\Brazilian Eccomerce Dataset\\product_category_name_translation.csv')

#### Merge on required datasets

In [219]:
merged_orders = pd.merge(orders_data,order_item_data, on='order_id')
merged_products = pd.merge(merged_orders, product_data, on ='product_id')
#separate date and time of oder purchase timestamp
date_split= merged_orders['order_purchase_timestamp'].str.split(' ')
date_split.head()

0    [2017-10-02, 10:56:33]
1    [2018-07-24, 20:41:37]
2    [2018-08-08, 08:38:49]
3    [2017-11-18, 19:28:06]
4    [2018-02-13, 21:18:39]
Name: order_purchase_timestamp, dtype: object

In [220]:
merged_orders['order_purchase_date']=date_split.str.get(0)
merged_orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,order_purchase_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,2017-10-02
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,2018-07-24
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,2018-08-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20,2017-11-18
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72,2018-02-13


In [221]:
# Creating column and shortening the product id
merged_orders["order_purchase_date"]= pd.to_datetime(merged_orders["order_purchase_date"])
merged_products['product_id_shorten']=merged_products['product_id'].str[-8:]

In [222]:
# Merge on Seller data
seller_products = pd.merge(order_item_data, sellers_data, on="seller_id")
seller_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   order_id                112650 non-null  object 
 1   order_item_id           112650 non-null  int64  
 2   product_id              112650 non-null  object 
 3   seller_id               112650 non-null  object 
 4   shipping_limit_date     112650 non-null  object 
 5   price                   112650 non-null  float64
 6   freight_value           112650 non-null  float64
 7   seller_zip_code_prefix  112650 non-null  int64  
 8   seller_city             112650 non-null  object 
 9   seller_state            112650 non-null  object 
dtypes: float64(2), int64(2), object(6)
memory usage: 9.5+ MB


In [223]:
date_split1= merged_products['order_purchase_timestamp'].str.split(' ')
merged_products['order_purchase_date']=date_split.str.get(0)

In [206]:
forecast = merged_products[['order_purchase_date','product_category_name','product_id_shorten','price']]
forecast["product_category_id"] = forecast.apply(lambda x: f"{x['product_category_name']}_{x['product_id_shorten']}", axis=1)
forecast1 = forecast
forecast2 = forecast1.dropna()
forecast3 = forecast2.groupby(['order_purchase_date', 'product_category_name'])['price'].mean().reset_index()

<ipython-input-206-d5458c165cd5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  forecast["product_category_id"] = forecast.apply(lambda x: f"{x['product_category_name']}_{x['product_id_shorten']}", axis=1)


In [209]:
forecast3.columns

Index(['order_purchase_date', 'product_category_name', 'price'], dtype='object')

In [212]:
list_df = ['utilidades_domesticas', 'perfumaria', 'automotivo', 'pet_shop',
       'papelaria', 'moveis_decoracao', 'moveis_escritorio',
       'ferramentas_jardim', 'informatica_acessorios', 'cama_mesa_banho',
       'brinquedos', 'construcao_ferramentas_construcao', 'telefonia',
       'beleza_saude', 'eletronicos', 'bebes', 'cool_stuff',
       'relogios_presentes', 'climatizacao', 'esporte_lazer',
       'livros_interesse_geral', 'eletroportateis', 'alimentos',
       'malas_acessorios', 'fashion_underwear_e_moda_praia',
       'artigos_de_natal', 'fashion_bolsas_e_acessorios',
       'instrumentos_musicais', 'construcao_ferramentas_iluminacao',
       'livros_tecnicos', 'construcao_ferramentas_jardim',
       'eletrodomesticos', 'market_place', 'agro_industria_e_comercio',
       'artigos_de_festas', 'casa_conforto', 'cds_dvds_musicais',
       'industria_comercio_e_negocios', 'consoles_games', 'moveis_quarto',
       'construcao_ferramentas_seguranca', 'telefonia_fixa', 'bebidas',
       'moveis_cozinha_area_de_servico_jantar_e_jardim',
       'fashion_calcados', 'casa_construcao', 'audio',
       'eletrodomesticos_2', 'fashion_roupa_masculina', 'cine_foto',
       'moveis_sala', 'artes', 'alimentos_bebidas',
       'tablets_impressao_imagem', 'fashion_esporte',
       'portateis_cozinha_e_preparadores_de_alimentos', 'la_cuisine',
       'flores', 'pcs', 'casa_conforto_2', 'portateis_casa_forno_e_cafe',
       'dvds_blu_ray', 'pc_gamer', 'construcao_ferramentas_ferramentas',
       'fashion_roupa_feminina', 'moveis_colchao_e_estofado',
       'sinalizacao_e_seguranca', 'fraldas_higiene', 'livros_importados',
       'fashion_roupa_infanto_juvenil', 'musica', 'artes_e_artesanato',
       'seguros_e_servicos']

In [217]:
# Fuction to extract data on product categories
def extract_df(list_df, forecast3):
    for i in list_df:
        new_df = forecast3[forecast3['product_category_name'] == i]
        new_df['order_purchase_date'] = new_df['order_purchase_date'].str[:7]
        new_df = new_df.groupby('order_purchase_date')['price'].mean().reset_index()
        new_df['order_purchase_date']=pd.to_datetime(new_df['order_purchase_date'], errors='coerce')
        new_df['product_category_name'] = i
        new_df = new_df[['order_purchase_date','product_category_name','price']]
        new_df.to_csv('C:\\Users\\Ajit Wagh\\Desktop\\TimeSeries\\new_categories\\new_files\\monthly_product_category_input\\'+str(i)+'.csv', index=False)
        print("length of " +str(i)+":", len(new_df))
        
extract_df(list_df, forecast3)

<ipython-input-217-c0c0492bb073>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['order_purchase_date'] = new_df['order_purchase_date'].str[:7]


length of utilidades_domesticas: 21
length of perfumaria: 21
length of automotivo: 21
length of pet_shop: 21
length of papelaria: 21
length of moveis_decoracao: 22
length of moveis_escritorio: 21
length of ferramentas_jardim: 22
length of informatica_acessorios: 22
length of cama_mesa_banho: 22
length of brinquedos: 21
length of construcao_ferramentas_construcao: 21
length of telefonia: 21
length of beleza_saude: 21
length of eletronicos: 21
length of bebes: 21
length of cool_stuff: 21
length of relogios_presentes: 21
length of climatizacao: 21
length of esporte_lazer: 22
length of livros_interesse_geral: 21
length of eletroportateis: 20
length of alimentos: 20
length of malas_acessorios: 21
length of fashion_underwear_e_moda_praia: 20
length of artigos_de_natal: 19
length of fashion_bolsas_e_acessorios: 21
length of instrumentos_musicais: 21
length of construcao_ferramentas_iluminacao: 21
length of livros_tecnicos: 20
length of construcao_ferramentas_jardim: 20
length of eletrodomesti